# Dataset Preparation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AustralianCancerDataNetwork/pydicer/blob/main/examples/DatasetPreparation.ipynb)

When working with real-world DICOM datasets, you will often need to tackle the task of cleaning the
dataset. Often you will have several image series, structure set and even dose grids for each
patient. However you typically want to select one relevant DICOM object in each category.

To help solve this, PyDicer provides a `dataset preparation` module which can be used to extract
a subset of data from your overall set. Two example use cases where this might be useful are:

**Analysing dose to structures for a radiotherapy treatment**
You will want to extract the dose grid which was calculated from the plan used to treat the
patient, as well as the linked structure set and planning CT image.

**Validating an Auto-segmentation tool**
A structure set may have been prepared for the purposes of validation and saved off with a specific
`SeriesDescription`. You select the latest structure set with that description as well as the
linked image series to perform the auto-segmentation validation.

As you will see in the examples below, you can provide your own logic to extract subsets of data
using PyDicer.

In [ ]:
try:
    from pydicer import PyDicer
except ImportError:
    !pip install pydicer
    from pydicer import PyDicer

from pathlib import Path
import pandas as pd

from pydicer.utils import fetch_converted_test_data

## Setup PyDicer

As in some other examples, we will use the HNSCC data prepared which has been preprepared and is
downloaded into the `testdata_hnscc` directory. We also setup our `PyDicer` object.

In [ ]:
working_directory = fetch_converted_test_data("./testdata_hnscc", dataset="HNSCC")

pydicer = PyDicer(working_directory)

## Explore data

When we use the `read_converted_data` function, by default it will return all data which has been
converted and is stored in the `testdata_hnscc/data` directory.

Let's use this function and output the entire DataFrame of converted data to see what we have
available in this dataset.

In [ ]:
df = pydicer.read_converted_data()
df

## Prepare dose data

Here we use the `dataset preparation` module to extract the latest dose grid by date. We refer to
this subset of data as `dose_project`.

We use the built in data extraction function, named [rt_latest_dose](https://australiancancerdatanetwork.github.io/pydicer/dataset.html#pydicer.dataset.functions.rt_latest_dose).

In [ ]:
dose_project_name = "dose_project"
pydicer.dataset.prepare(dose_project_name, "rt_latest_dose")

Once the cell above has finished running, the dataset has been prepared. You can explore the
dataset in the `testdata_hnscc/dose_project` directory. Take notice of two things:
- The `converted.csv` file stored from each patient now only includes the data objects which have
  been selected as part of this subset of data.
- The data object folders are not actual folders, but symbolic links to the original data found in
  the `testdata_hnscc/data` directory. Like this, data isn't duplicated but the folder structure
  remains easy to navigate.

> Note: Symbolic links are supported on Unix-based (Linux, MacOS) operating systems only. These
> won't work on Windows however you can still use the dataset prepared which is tracked in the
> converted csv files.

## Load prepared Dataset

By supplying the `dataset_name` to the `read_converted_data` function, we obtain a DataFrame
containing only the data objects part of that subset.

In [ ]:
df_dose_project = pydicer.read_converted_data(dataset_name=dose_project_name)
df_dose_project

Notice that we now only have one of each data object modality in our `dose_project` subset. We are
now ready to work with that subset (e.g. extract dose metrics).

## Prepare Structure Dataset

In the next example, we are only want to extract structure sets and their associated images. This
might be useful when training or validating an auto-segmentation model.

In this example, we not only select the latest structure set by date, but we specify the
`StudyDescription` values of the DICOM metadata of the data objects we want to select. To achieve
this, we use the build in [rt_latest_struct](https://australiancancerdatanetwork.github.io/pydicer/dataset.html#pydicer.dataset.functions.rt_latest_struct) function.

Observe the output of the following cell and explore the `testdata_hnscc` directory. We not have
one structure set and the linked image for each patient.

In [ ]:
# Define the dataset name and the study description values to match
structure_project_name = "structure_project"
series_descriptions = [
    "RT SIMULATION"
]


# Prepare the subset of data
pydicer.dataset.prepare(
    structure_project_name,
    "rt_latest_struct",
    StudyDescription=series_descriptions
)

# Load the data subset and display the DataFrame
df_structure_project = pydicer.read_converted_data(dataset_name=structure_project_name)
df_structure_project

## Prepare Dataset from DataFrame

In some scenarios, you may want to simply perform some filtering on the DataFrame returned by the
`read_converted_data` function and generate a subset of data based on that.

In the following cell, a subset of data named `image_project` is generated by filtering the
DataFrame to keep only `CT` images.

After running the following cell, explore the `testdata_hnscc/image_project` directory to confirm
that only image objects were selected.

In [ ]:
# Read the converted DataFrame and filter only CT images
df = pydicer.read_converted_data()
df_ct = df[df.modality=="CT"]

# Prepare a data subset using this filtered DataFrame
image_project_name = "image_project"
pydicer.dataset.prepare_from_dataframe(image_project_name, df_ct)

# Load the data subset and display the DataFrame
df_image_project = pydicer.read_converted_data(dataset_name=image_project_name)
df_image_project


## Define Custom Preparation Function

In more complex use cases you may want to define your own logic for extracting data objects into a
subset. For example, you may have an additional DataFrame containing treatment start dates of
patients, and you would like to select the dose grid, structure set and image series which are
closest to that date.

In the following cell, we preare a `clinical_project` subset of data. We create a dummy set of
clinical tabular data `df_clinical`. This stores each patient's stage and RT start date.

We use the information in `df_clinical`, to select patients who are stage 1-3 along with the data
objects where the dose grid date is nearest to their treatment start date.

In [ ]:
# Define some dummy clinical data
df_clinical = pd.DataFrame([
    {
        "patient_id": "HNSCC-01-0199",
        "stage": 2,
        "rt_start_date": "2002-10-28",
    },
    {
        "patient_id": "HNSCC-01-0176",
        "stage": 1,
        "rt_start_date": "2009-03-02",
    },
    {
        "patient_id": "HNSCC-01-0019",
        "stage": 4,
        "rt_start_date": "1998-07-10",
    },
])

# Convert date to a datetime object
df_clinical['rt_start_date'] = pd.to_datetime(df_clinical['rt_start_date'], format='%Y-%m-%d')

In [ ]:
# Import some pydicer utility functions that we'll need
from pydicer.utils import load_object_metadata, determine_dcm_datetime

# Define a function which accept the converted DataFrame as input and returns a filtered DataFrame
# of objects to keep in the data subset. This function also takes the clinical DataFrame as input.
def extract_clinical_data(df_data, df_clinical):

    # Merge the clinical data with our data objects
    df = pd.merge(df_data, df_clinical, on="patient_id", how="outer")

    # Filter out patients who aren't stage 1-3
    df = df[(df.stage >= 1) & (df.stage <= 3)]

    # Determine the date of each data object
    df["obj_date"] = df.apply(lambda row: determine_dcm_datetime(load_object_metadata(row)), axis=1)

    # List to track row indicies we will keep
    keep_rows = []

    # Sort their dose grids by descending order, so we can select the first (latest)
    # dose grid and link the structure set and image series to use for the data subset.
    df = df.sort_values("obj_date", ascending=False)

    # Loop the data by patient to select the data objects
    for patient_id, df_pat in df.groupby("patient_id"):

        df_doses = df_pat[df_pat.modality=="RTDOSE"]

        # If there are no dose grid, we skip this patient
        if len(df_doses) == 0:
            continue

        # Otherwise, we select the first dose grid (which is the latest since they are sorted) 
        # to keep
        dose_row = df_doses.iloc[0]

        df_linked_structs = pydicer.get_structures_linked_to_dose(dose_row)

        # Skip patient if no linked structure sets are found
        if len(df_linked_structs) == 0:
            continue

        # Finally, find the image linked to the structure set
        struct_row = df_linked_structs.iloc[0]

        df_linked_images = df[df.sop_instance_uid==struct_row.referenced_sop_instance_uid]

        # Skip if no images found
        if len(df_linked_images) == 0:
            continue

        image_row = df_linked_images.iloc[0]

        # Store the indcies of these data objects
        keep_rows.append(image_row.name)
        keep_rows.append(struct_row.name)
        keep_rows.append(dose_row.name)

    # Return only the rows of the data objects we want to keep in the data subset
    return df_data.loc[keep_rows]

In [ ]:
clinical_project_name = "clinical"

# Prepare the subset of data using our custom function
pydicer.dataset.prepare(
    clinical_project_name,
    extract_clinical_data,
    df_clinical=df_clinical
)

# Load the data subset and display the DataFrame
df_clinical_project = pydicer.read_converted_data(dataset_name=clinical_project_name)
df_clinical_project